In [1]:
# let's try training a model to predict arrival time for one month.

In [1]:
from traffic.core import Traffic
from preprocessing import get_complete_flights, preprocess_traffic
import h5py
import numpy as np
import pandas as pd
import datetime

In [15]:
# let's first practice with three days
file = "Frankfurt_LH_2206.h5"

with h5py.File(file, 'r') as f:
    keys = (list(f.keys()))
    for i in range(3):
        key = keys[i]
        if i == 0:
            flights = Traffic.from_file(file, key = key, parse_dates=["day","firstseen", "hour", "last_position","lastseen","timestamp"])
        else:
            flights_add = Traffic.from_file(file, key = key, parse_dates=["day","firstseen", "hour", "last_position","lastseen","timestamp"])
            flights += flights_add

In [17]:
df_flights = preprocess_traffic(flights)

In [2]:
#create feature as seconds till landing
def seconds_till_arrival(flights_data: pd.DataFrame):
    time_till_arrival = flights_data["arrival_time"]- flights_data["timestamp"]
    seconds = time_till_arrival.dt.total_seconds()
    return seconds

In [3]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.models import load_model
from sklearn.preprocessing import StandardScaler

In [4]:
model = load_model("model5")

In [5]:
def build_sequential(lr, input_dims, output_dims, layerSizes, activation, loss):
    model = keras.Sequential()
    model.add (keras.layers.Input(shape=input_dims))
    for size in layerSizes:
        model.add(keras.layers.Dense(size))
        model.add(keras.layers.LeakyReLU(alpha=0.05))

    model.add(keras.layers.Dense(output_dims,activation=activation))

    model.compile(optimizer=Adam(learning_rate=lr), loss=loss)

    return model

In [ ]:

months = ["01","02", "03", "05", "06", "07", "08", "09", "10", "11"]
for j in range(10):
    print(j)
    first = True
    for month in months:
        file = "Frankfurt_LH_22" + month + ".h5"
        for day in range(1,32):

            if month in ["06", "09", "11"] and day ==31:
                continue
            str_day = str(day) if day > 9 else "0"+str(day)
            key = "LH_22"+month+str_day
            with h5py.File(file, 'r') as f:
                print(key)
                if key in list(f.keys()):
                    flights = Traffic.from_file(file, key = key, parse_dates=["day","firstseen", "hour", "last_position","lastseen","timestamp"])
                else:
                    continue
                try:
                    df_flights = preprocess_traffic(flights)
                except:
                    continue
                df = df_flights[["distance", "altitude", "geoaltitude", "arrival_time", "timestamp", "vertical_rate", "groundspeed"]].dropna()
                df_sample = df.sample(frac=0.05)
                if not first:
                    df_train = pd.concat([df_sample, df_train])
                else:
                    df_train = df_sample
                    first = False

    X = np.array(df_train[["distance", "altitude", "geoaltitude", "vertical_rate", "groundspeed"]])  # we only take the feature
    y = seconds_till_arrival(df_train)
    if j == 0:
        scaler = StandardScaler()
        scaler = scaler.fit(X)
        #model = build_sequential(lr=0.0001, input_dims=(X.shape[1],),output_dims=1, layerSizes=(256,128,64,32,16,8,4,2),activation=None,loss="MSE")
        X_scaled = scaler.transform(X)
        model.fit(X_scaled,y, epochs = 7)
    else:
        X_scaled = scaler.transform(X)
        print("evaluating model")
        model.evaluate(X_scaled,y)
        print("training_model")
        model.fit(X_scaled,y, epochs = 7)

        
    model.save("model"+str(j))







0
LH_220101
LH_220102
LH_220103
LH_220104
LH_220105
LH_220106
LH_220107
LH_220108
LH_220109
LH_220110
LH_220111
LH_220112
LH_220113
LH_220114
LH_220115
LH_220116
LH_220117
LH_220118
LH_220119
LH_220120
LH_220121
LH_220122
LH_220123
LH_220124
LH_220125
LH_220126
LH_220127
LH_220128
LH_220129
LH_220130
LH_220131
LH_220201
LH_220202
LH_220203
LH_220204
LH_220205
LH_220206
LH_220207
LH_220208
LH_220209
LH_220210
LH_220211
LH_220212
LH_220213
LH_220214
LH_220215
LH_220216
LH_220217
LH_220218
LH_220219
LH_220220
LH_220221
LH_220222
LH_220223
LH_220224
LH_220225
LH_220226
LH_220227
LH_220228
LH_220229
LH_220230
LH_220231
LH_220301
LH_220302
LH_220303
LH_220304
LH_220305
LH_220306
LH_220307
LH_220308
LH_220309
LH_220310
LH_220311
LH_220312
LH_220313
LH_220314
LH_220315
LH_220316
LH_220317
LH_220318
LH_220319
LH_220320
LH_220321
LH_220322
LH_220323
LH_220324
LH_220325
LH_220326
LH_220327
LH_220328
LH_220329
LH_220330
LH_220331
LH_220501
LH_220502
LH_220503
LH_220504
LH_220505
LH_220506
LH_22050

INFO:tensorflow:Assets written to: model0\assets


INFO:tensorflow:Assets written to: model0\assets


1
LH_220101
LH_220102
LH_220103
LH_220104
LH_220105
LH_220106
LH_220107
LH_220108
LH_220109
LH_220110
LH_220111
LH_220112
LH_220113
LH_220114
LH_220115
LH_220116
LH_220117
LH_220118
LH_220119
LH_220120
LH_220121
LH_220122
LH_220123
LH_220124
LH_220125
LH_220126
LH_220127
LH_220128
LH_220129
LH_220130
LH_220131
LH_220201
LH_220202
LH_220203
LH_220204
LH_220205
LH_220206
LH_220207
LH_220208
LH_220209
LH_220210
LH_220211
LH_220212
LH_220213
LH_220214
LH_220215
LH_220216
LH_220217
LH_220218
LH_220219
LH_220220
LH_220221
LH_220222
LH_220223
LH_220224
LH_220225
LH_220226
LH_220227
LH_220228
LH_220229
LH_220230
LH_220231
LH_220301
LH_220302
LH_220303
LH_220304
LH_220305
LH_220306
LH_220307
LH_220308
LH_220309
LH_220310
LH_220311
LH_220312
LH_220313
LH_220314
LH_220315
LH_220316
LH_220317
LH_220318
LH_220319
LH_220320
LH_220321
LH_220322
LH_220323
LH_220324
LH_220325
LH_220326
LH_220327
LH_220328
LH_220329
LH_220330
LH_220331
LH_220501
LH_220502
LH_220503
LH_220504
LH_220505
LH_220506
LH_22050

INFO:tensorflow:Assets written to: model1\assets


...

INFO:tensorflow:Assets written to: model3\assets


4
LH_220101
LH_220102
LH_220103
LH_220104
LH_220105
LH_220106
LH_220107
LH_220108
LH_220109
LH_220110
LH_220111
LH_220112
LH_220113
LH_220114
LH_220115
LH_220116
LH_220117
LH_220118
LH_220119
LH_220120
LH_220121
LH_220122
LH_220123
LH_220124
LH_220125
LH_220126
LH_220127
LH_220128
LH_220129
LH_220130
LH_220131
LH_220201
LH_220202
LH_220203
LH_220204
LH_220205
LH_220206
LH_220207
LH_220208
LH_220209
LH_220210
LH_220211
LH_220212
LH_220213
LH_220214
LH_220215
LH_220216
LH_220217
LH_220218
LH_220219
LH_220220
LH_220221
LH_220222
LH_220223
LH_220224
LH_220225
LH_220226
LH_220227
LH_220228
LH_220229
LH_220230
LH_220231
LH_220301
LH_220302
LH_220303
LH_220304
LH_220305
LH_220306
LH_220307
LH_220308
LH_220309
LH_220310
LH_220311
LH_220312
LH_220313
LH_220314
LH_220315
LH_220316
LH_220317
LH_220318
LH_220319
LH_220320
LH_220321
LH_220322
LH_220323
LH_220324
LH_220325
LH_220326
LH_220327
LH_220328
LH_220329
LH_220330
LH_220331
LH_220501
LH_220502
LH_220503
LH_220504
LH_220505
LH_220506
LH_22050

INFO:tensorflow:Assets written to: model4\assets


INFO:tensorflow:Assets written to: model4\assets


5
LH_220101
LH_220102
LH_220103
LH_220104
LH_220105
LH_220106
LH_220107
LH_220108
LH_220109
LH_220110
LH_220111
LH_220112
LH_220113
LH_220114
LH_220115
LH_220116
LH_220117
LH_220118
LH_220119
LH_220120
LH_220121
LH_220122
LH_220123
LH_220124
LH_220125
LH_220126
LH_220127
LH_220128
LH_220129
LH_220130
LH_220131
LH_220201
LH_220202
LH_220203
LH_220204
LH_220205
LH_220206
LH_220207
LH_220208
LH_220209
LH_220210
LH_220211
LH_220212
LH_220213
LH_220214
LH_220215
LH_220216
LH_220217
LH_220218
LH_220219
LH_220220
LH_220221
LH_220222
LH_220223
LH_220224
LH_220225
LH_220226
LH_220227
LH_220228
LH_220229
LH_220230
LH_220231
LH_220301
LH_220302
LH_220303
LH_220304
LH_220305
LH_220306
LH_220307
LH_220308
LH_220309
LH_220310
LH_220311
LH_220312
LH_220313
LH_220314
LH_220315
LH_220316
LH_220317
LH_220318
LH_220319
LH_220320
LH_220321
LH_220322
LH_220323
LH_220324


In [9]:
X = np.array(df_train[["distance", "altitude", "geoaltitude", "vertical_rate", "groundspeed"]])  # we only take the feature
y = seconds_till_arrival(df_train)
if j == 0:
    scaler = StandardScaler()
    scaler = scaler.fit(X)
    #model = build_sequential(lr=0.0001, input_dims=(X.shape[1],),output_dims=1, layerSizes=(256,128,64,32,16,8,4,2),activation=None,loss="MSE")
    X_scaled = scaler.transform(X)
    model.fit(X_scaled,y, epochs = 7)

Epoch 1/7
313422/313422 [==============================] - 607s 2ms/step - loss: 149800.1094
Epoch 2/7
313422/313422 [==============================] - 665s 2ms/step - loss: 148186.8594
Epoch 3/7
313422/313422 [==============================] - 654s 2ms/step - loss: 146713.5000
Epoch 4/7
313422/313422 [==============================] - 636s 2ms/step - loss: 145273.1406
Epoch 5/7
313422/313422 [==============================] - 697s 2ms/step - loss: 143921.2656
Epoch 6/7
300837/313422 [===========================>..] - ETA: 25s - loss: 142872.0938